In [1]:
from src.api_call import query_endpoint, datetime_to_unix, headers, extract_reading_data,load_assets_from_json
from src.temp_pre import DigitalTwinModel

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import pandas as pd
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
import re

## 1. Data Acquisition (api_call.py)
* Taking W16 as an example, extract the data from the last 24 hours.
* I'm not sure how the data is retrieved within the app, but we can make the necessary adjustments accordingly.

In [2]:
assets = [
    {"id": 9274, "name": "WONING 16 - digitale meter"},
    {"id": 9825, "name": "WONING 16 - Badkamer"},
    {"id": 9834, "name": "WONING 16 - Eetkamer"},
    {"id": 9832, "name": "WONING 16 - Hal beneden"},
    {"id": 15481, "name": "WONING 16 - Hal boven"},
    {"id": 9826, "name": "WONING 16 - Keuken"},
    {"id": 9267, "name": "WONING 16 - Koelkast"},
    {"id": 9272, "name": "WONING 16 - Living"},
    {"id": 9269, "name": "WONING 16 - slaapkamer 1"},
    {"id": 9270, "name": "WONING 16 - slaapkamer 2"},
    {"id": 9271, "name": "WONING 16 - slaapkamer 3"},
    {"id": 9268, "name": "WONING 16 - TV"},
    {"id": 9266, "name": "WONING 16 - Wasmachine"},
    {"id": 9273, "name": "WONING 16 - watermeter"}
]
# 1. Define time range
end_time = datetime.now(ZoneInfo("UTC"))
start_time = end_time - timedelta(hours=24)

print(f"Fetching data from {start_time} to {end_time}...")

df = pd.DataFrame()

# 2. Iterate through assets
for asset in assets:
    asset_id = asset['id']
    asset_name = asset['name']
    
    # Create a clean prefix from the asset name (e.g., "Slaapkamer_1")
    # This replaces spaces/dashes with underscores and removes special characters
    clean_prefix = re.sub(r'[^\w\s]', '', asset_name).strip().replace(' ', '_')
    
    try:
        response = query_endpoint('aggregateseries', header=headers, assetid=asset_id, 
                                  start_time=start_time, end_time=end_time, dry_run=False)
        
        reading_data = extract_reading_data(response, asset_id)
        if not reading_data:
            continue
            
        df_temp = pd.DataFrame(reading_data)
        df_temp = df_temp.groupby('Timestamp').agg('first').reset_index()
        df_temp['Timestamp'] = pd.to_datetime(df_temp['Timestamp'])

        # Drop internal columns
        cols_to_drop = [c for c in ['SensorID', 'SensorType'] if c in df_temp.columns]
        df_temp = df_temp.drop(columns=cols_to_drop)

        # RENAME columns with prefix to avoid duplicate suffixes (e.g., "Slaapkamer_1_temperature")
        # Timestamp is excluded from renaming so it can be used for merging
        new_cols = {col: f"{clean_prefix}_{col}" for col in df_temp.columns if col != 'Timestamp'}
        df_temp = df_temp.rename(columns=new_cols)

        # 3. Merge
        if df.empty:
            df = df_temp
        else:
            df = pd.merge(df, df_temp, on='Timestamp', how='outer')
            
        print(f"Success: {asset_name}")
        
    except Exception as e:
        print(f"Failed to fetch {asset_name}: {e}")

# 4. Final Processing
if not df.empty:
    df = df.sort_values('Timestamp').reset_index(drop=True)
    df = df.interpolate(method='linear').ffill().bfill()
    print(f"\nMerge complete. Shape: {df.shape}")
else:
    print("\nNo data retrieved.")
df.head()

Fetching data from 2026-02-03 19:21:49.336241+00:00 to 2026-02-04 19:21:49.336241+00:00...
Success: WONING 16 - digitale meter
Success: WONING 16 - Badkamer
Success: WONING 16 - Eetkamer
Success: WONING 16 - Hal beneden
Success: WONING 16 - Hal boven
Success: WONING 16 - Keuken
Success: WONING 16 - Koelkast
Success: WONING 16 - Living
Success: WONING 16 - slaapkamer 1
Success: WONING 16 - slaapkamer 2
Success: WONING 16 - slaapkamer 3
Success: WONING 16 - TV
Success: WONING 16 - Wasmachine
Success: WONING 16 - watermeter

Merge complete. Shape: (721, 113)


,Timestamp,WONING_16__digitale_meter_current_tariff,WONING_16__digitale_meter_gas.kuub,WONING_16__digitale_meter_negative_active_power,WONING_16__digitale_meter_phase_a.current,WONING_16__digitale_meter_phase_a.negative_active_power,WONING_16__digitale_meter_phase_a.positive_active_power,WONING_16__digitale_meter_positive_active_power,WONING_16__digitale_meter_tariff1.negative_active_energy,WONING_16__digitale_meter_tariff1.positive_active_energy,WONING_16__digitale_meter_tariff2.negative_active_energy,WONING_16__digitale_meter_tariff2.positive_active_energy,WONING_16__Badkamer_battery,WONING_16__Badkamer_light_level,WONING_16__Badkamer_pir_status,WONING_16__Badkamer_motor.position,WONING_16__Badkamer_motor.stroke,WONING_16__Badkamer_temperature,WONING_16__Badkamer_temperature.set,WONING_16__Eetkamer_battery,WONING_16__Eetkamer_light_level,WONING_16__Eetkamer_pir_status,WONING_16__Hal_beneden_accMotion,WONING_16__Hal_beneden_digital,WONING_16__Hal_beneden_door.status,WONING_16__Hal_beneden_pulseAbs,WONING_16__Hal_beneden_vdd,WONING_16__Hal_beneden_x,WONING_16__Hal_beneden_y,WONING_16__Hal_beneden_z,WONING_16__Hal_beneden_battery,WONING_16__Hal_beneden_motor.position,WONING_16__Hal_beneden_motor.stroke,WONING_16__Hal_beneden_temperature,WONING_16__Hal_beneden_temperature.set,WONING_16__Hal_boven_battery,WONING_16__Hal_boven_light_level,WONING_16__Hal_boven_pir_status,WONING_16__Keuken_battery,WONING_16__Keuken_light_level,WONING_16__Keuken_pir_status,WONING_16__Keuken_motor.position,WONING_16__Keuken_motor.stroke,WONING_16__Keuken_temperature,WONING_16__Keuken_temperature.set,WONING_16__Koelkast_active_power,WONING_16__Koelkast_current,WONING_16__Koelkast_state,WONING_16__Koelkast_total_active_energy,WONING_16__Koelkast_voltage,WONING_16__Living_battery,WONING_16__Living_buzzer_status,WONING_16__Living_co2,WONING_16__Living_humidity,WONING_16__Living_light_level,WONING_16__Living_pir_status,WONING_16__Living_power_on,WONING_16__Living_pressure,WONING_16__Living_temperature,WONING_16__Living_tvoc,WONING_16__slaapkamer_1_accMotion,WONING_16__slaapkamer_1_digital,WONING_16__slaapkamer_1_door.status,WONING_16__slaapkamer_1_pulseAbs,WONING_16__slaapkamer_1_vdd,WONING_16__slaapkamer_1_x,WONING_16__slaapkamer_1_y,WONING_16__slaapkamer_1_z,WONING_16__slaapkamer_1_battery,WONING_16__slaapkamer_1_motor.position,WONING_16__slaapkamer_1_motor.stroke,WONING_16__slaapkamer_1_temperature,WONING_16__slaapkamer_1_temperature.set,WONING_16__slaapkamer_2_accMotion,WONING_16__slaapkamer_2_digital,WONING_16__slaapkamer_2_door.status,WONING_16__slaapkamer_2_pulseAbs,WONING_16__slaapkamer_2_vdd,WONING_16__slaapkamer_2_x,WONING_16__slaapkamer_2_y,WONING_16__slaapkamer_2_z,WONING_16__slaapkamer_2_battery,WONING_16__slaapkamer_2_motor.position,WONING_16__slaapkamer_2_motor.stroke,WONING_16__slaapkamer_2_temperature,WONING_16__slaapkamer_2_temperature.set,WONING_16__slaapkamer_3_accMotion,WONING_16__slaapkamer_3_digital,WONING_16__slaapkamer_3_door.status,WONING_16__slaapkamer_3_pulseAbs,WONING_16__slaapkamer_3_vdd,WONING_16__slaapkamer_3_x,WONING_16__slaapkamer_3_y,WONING_16__slaapkamer_3_z,WONING_16__slaapkamer_3_battery,WONING_16__slaapkamer_3_motor.position,WONING_16__slaapkamer_3_motor.stroke,WONING_16__slaapkamer_3_temperature,WONING_16__slaapkamer_3_temperature.set,WONING_16__TV_active_power,WONING_16__TV_current,WONING_16__TV_state,WONING_16__TV_total_active_energy,WONING_16__TV_voltage,WONING_16__Wasmachine_active_power,WONING_16__Wasmachine_current,WONING_16__Wasmachine_state,WONING_16__Wasmachine_total_active_energy,WONING_16__Wasmachine_voltage,WONING_16__watermeter_battery,WONING_16__watermeter_humidity,WONING_16__watermeter_pulsecounter.pulses,WONING_16__watermeter_temperature
0,2026-02-03 19:20:00+00:00,2.0,9233.717,0.0,4.0,0.0,0.933,0.933,1692.22,10117.444,3849.103,9346.320,80.0,0.0,0.0,0.0,498.0,16.3,19.0,55.0,0.0,0.0,0.0,1.0,NaN,1965.0,3629.0,-1.0,61.0,-2.0,81.0,512.0,512.0,15.7,16.0,68.0,0.0,0.0,66.0,0.0,0.0,0.0,506.0,17.1,20.0,41.0,2.

## The directly merged dataframe serves as the starting point for preprocessing.

## Module 1: Data Pre-processing & Cleaning

---

### **Cleaning Steps:**
* **Feature Selection:** We filter for **Temperature**, **Setpoints**, and **PIR (Occupancy)** status.
* **Noise Removal:** Strictly excluded **Watermeter** data as it does not influence room temperature.
* **Synchronization:** All sensors are aligned to a fixed **10-minute interval**.
* **Gap Filling:** Any missing points are filled using **linear interpolation** to ensure a continuous sequence.
* **Temporal Cycles:** Hour and Day are encoded into **Sine/Cosine waves** to help the model perceive time as a continuous loop.

In [3]:
# 1. Initialize
dt_model = DigitalTwinModel(lookback_steps=144, forecast_steps=18)

# 2. Process (Module 1)
clean_df = dt_model.prepare_clean_df(df)
clean_df.head()


,WONING_16__Badkamer_pir_status,WONING_16__Badkamer_temperature,WONING_16__Badkamer_temperature.set,WONING_16__Eetkamer_pir_status,WONING_16__Hal_beneden_temperature,WONING_16__Hal_beneden_temperature.set,WONING_16__Hal_boven_pir_status,WONING_16__Keuken_pir_status,WONING_16__Keuken_temperature,WONING_16__Keuken_temperature.set,WONING_16__Living_pir_status,WONING_16__Living_temperature,WONING_16__slaapkamer_1_temperature,WONING_16__slaapkamer_1_temperature.set,WONING_16__slaapkamer_2_temperature,WONING_16__slaapkamer_2_temperature.set,WONING_16__slaapkamer_3_temperature,WONING_16__slaapkamer_3_temperature.set,hour_sin,hour_cos,day_sin,day_cos
Timestamp,,,,,,,,,,,,,,,,,,,,,,
2026-02-03 20:20:00+01:00,0.0,16.300,19.0,1.0,15.700,16.0,0.0,0.0,17.10,20.0,0.0,18.10,14.90,13.0,14.70,15.0,14.700,13.0,-0.866025,0.500000,0.781831,0.62349
2026-02-03 20:30:00+01:00,0.0,16.276,19.0,0.0,15.712,16.0,0.0,0.0,17.16,20.0,0.0,18.10,14.94,13.0,14.70,15.0,14.700,13.0,-0.866025,0.500000,0.781831,0.62349
2026-02-03 20:40:00+01:00,0.0,16.204,19.0,1.0,15.776,16.0,0.0,0.0,17.08,20.0,0.0,18.04,15.00,13.0,14.64,15.0,14.724,13.0,-0.866025,0.500000,0.781831,0.62349
2026-02-03 20:50:00+01:00,0.0,16.200,19.0,1.0,15.712,16.0,0.0,0.0,16.94,20.0,0.0,17.88,15.00,13.0,14.66,15.0,14.772,13.0,-0.866025,0.500000,0.781831,0.62349
2026-02-03 21:00:00+01:00,0.0,16.200,19.0,1.0,15.700,16.0,0.0,0.0,16.84,20.0,0.0,17.68,14.96,13.0,14.64,15.0,14.728,13.0,-0.707107,0.707107,0.781831,0.62349


In [4]:
# 3. Vectorize (Module 2)
input_tensor = dt_model.dataframe_to_tensor(clean_df)
input_tensor.size()

torch.Size([144, 22])

In [5]:
# 4. Infer (Module 3)
# Note: Since the model is untrained, the values will be random for now
dt_model.init_network(model_path="./model/woning16_model.pth") 
forecast_result = dt_model.predict_future(input_tensor)

# Check the output
import json
print(json.dumps(forecast_result, indent=2))

✅ Weights loaded for 7 rooms.
{
  "meta": {
    "type": "Multi-Room Prediction (No Watermeter)",
    "horizon": "3 Hours",
    "resolution": "10 min"
  },
  "rooms": {
    "WONING_16__Badkamer_temperature": [
      {
        "offset_min": 10,
        "temp": 7.16
      },
      {
        "offset_min": 20,
        "temp": 9.35
      },
      {
        "offset_min": 30,
        "temp": 8.01
      },
      {
        "offset_min": 40,
        "temp": 12.12
      },
      {
        "offset_min": 50,
        "temp": 8.58
      },
      {
        "offset_min": 60,
        "temp": 9.8
      },
      {
        "offset_min": 70,
        "temp": 10.49
      },
      {
        "offset_min": 80,
        "temp": 10.81
      },
      {
        "offset_min": 90,
        "temp": 6.72
      },
      {
        "offset_min": 100,
        "temp": 9.99
      },
      {
        "offset_min": 110,
        "temp": 9.93
      },
      {
        "offset_min": 120,
        "temp": 10.27
      },
      {
        "